In [1]:
import pandas as pd
import numpy as np
from datetime import *
from dateutil.relativedelta import relativedelta as reladate

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose



## read csv


In [3]:
df=pd.read_csv('vnindex_with_usd_dji_gold_oil.csv')
df=df[['date','OT']]

df['date'] = pd.to_datetime(df['date'])
df

,date,OT
0,2015-01-05,544.45
1,2015-01-06,549.66
2,2015-01-07,552.05
3,2015-01-08,553.47
4,2015-01-09,569.73
...,...,...
2551,2025-03-27,1323.81
2552,2025-03-28,1317.46
2553,2025-03-31,1306.86
2554,2025-04-01,1317.33


# Prepare NeuralForecast

## PreTrain

In [4]:
from neuralforecast import NeuralForecast
from neuralforecast.models import TimeMixer, TSMixer, TSMixerx
from neuralforecast.losses.pytorch import MAE

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


## Prepare Data


In [6]:
df1 = df
df1 = df1.reset_index().rename(columns={'date': 'ds'})
df1['unique_id'] = 'VNINDEX'
df1.rename(columns={'OT': 'y'}, inplace=True)
df1['ds'] = pd.to_datetime(df1['ds'])
# Kết quả
df1 = df1[['unique_id', 'ds', 'y']]
df1

,unique_id,ds,y
0,VNINDEX,2015-01-05,544.45
1,VNINDEX,2015-01-06,549.66
2,VNINDEX,2015-01-07,552.05
3,VNINDEX,2015-01-08,553.47
4,VNINDEX,2015-01-09,569.73
...,...,...,...
2551,VNINDEX,2025-03-27,1323.81
2552,VNINDEX,2025-03-28,1317.46
2553,VNINDEX,2025-03-31,1306.86
2554,VNINDEX,2025-04-01,1317.33


In [7]:
# Xác định kích thước các tập
train_size = int(len(df1) * 0.7)  # 70% cho train
val_len = int(len(df1) * 0.2)   # 20% cho validation
test_len = len(df1) - train_size - val_len  # Phần còn lại cho test

# Chia dữ liệu theo thời gian
train = df1.iloc[:train_size].reset_index(drop=True)
val = df1.iloc[train_size:train_size + val_len].reset_index(drop=True)
test = df1.iloc[train_size + val_len:].reset_index(drop=True)

# Kiểm tra kích thước
print("VNINDEX:")
print(f"Train size: {len(train)}")
print(f"Validation size: {len(val)}")
print(f"Test size: {len(test)}")


VNINDEX:
Train size: 1789
Validation size: 511
Test size: 256


# TimeMixer



## Bayes

In [7]:
### Bayes
import pandas as pd
import numpy as np
from skopt import gp_minimize, gbrt_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from neuralforecast import NeuralForecast
from neuralforecast.models import TimeMixer
from neuralforecast.losses.pytorch import MAE, MSE, RMSE
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
import warnings

warnings.filterwarnings('ignore')

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [21]:

# Assuming df, val_len, test_len are defined elsewhere
horizon = 20

# Define search space for TimeMixer hyperparameters
dimensions = [
    Integer(low=40, high=80, name='input_size'),  # Bắt đầu từ mẫu: input_size=24
    Integer(low=200, high=500, name='max_steps'),  # Bắt đầu từ mẫu: max_steps=500
    Real(low=1e-4, high=1e-3, prior='log-uniform', name='learning_rate'),  # Bắt đầu từ mẫu: learning_rate=1e-3
    Categorical(categories=[32, 64], name='batch_size'),  
    Integer(low=1, high=3, name='e_layers'),
    Categorical(categories=[256, 512], name='d_ff'),
    Real(low=0.0, high=0.3, name='dropout'),
]

# Global variables to store results
results_history = []
best_score = float('inf')
best_params = None
best_method = None
best_result = None

# Callback function to store optimization results
def callback_func(result):
    global results_history, best_score, best_params, best_method, best_result
    
    current_score = result.func_vals[-1]
    current_params = dict(zip([dim.name for dim in dimensions], result.x_iters[-1]))
    
    results_history.append({
        'iteration': len(result.func_vals),
        'score': current_score,
        **current_params
    })
    
    if current_score < best_score:
        best_score = current_score
        best_params = current_params.copy()
        best_result = result
        print(f"New best score: {current_score:.4f} at iteration {len(result.func_vals)}")

@use_named_args(dimensions)
def objective(**params):
    """
    Objective function for Bayesian Optimization
    """
    try:
# Tạo mô hình TimeMixer với các siêu tham số
        model = TimeMixer(
            h=horizon,
            input_size=params['input_size'],
            n_series=1,  # Lấy từ mẫu
            scaler_type='standard',  # Lấy từ mẫu
            max_steps=params['max_steps'],
            early_stop_patience_steps=5,  # Lấy từ mẫu
            val_check_steps=5,  # Lấy từ mẫu
            learning_rate=params['learning_rate'],
            loss=MAE(),  # Lấy từ mẫu
            valid_loss=MAE(),  # Lấy từ mẫu
            batch_size=params['batch_size'],           
            e_layers=params['e_layers'],
            d_ff=params['d_ff'],
            dropout=params['dropout'],
        )
        
        # Initialize NeuralForecast
        fcst = NeuralForecast(models=[model], freq='B')
        
        # Perform cross-validation
        forecasts = fcst.cross_validation(
            df=df1, 
            val_size=val_len, 
            n_windows=None, 
            test_size=test_len - ((test_len - horizon) % horizon), 
            step_size=horizon
        )
        
        # Calculate MAE
        Y_true = forecasts['y'].values
        Y_hat = forecasts['TimeMixer'].values
        MAE_score = mae(Y_true, Y_hat)
        
        print(f"MAE: {MAE_score:.4f} | Params: {params}")
        return MAE_score
        
    except Exception as e:
        print(f"Error in objective function: {e}")
        return 1000.0  # Penalty for failed runs



In [22]:

# Method 1: Gaussian Process Optimization
print("="*60)
print("PHƯƠNG PHÁP 1: GAUSSIAN PROCESS OPTIMIZATION")
print("="*60)

results_history = []  # Reset history
result_gp = gp_minimize(
    func=objective,
    dimensions=dimensions,
    n_calls=50,
    n_initial_points=10,
    acq_func='EI',
    random_state=42,
    callback=callback_func
)

print(f"Best GP score: {result_gp.fun:.4f}")
print("Best GP parameters:")
for dim, val in zip(dimensions, result_gp.x):
    print(f"  {dim.name}: {val}")
if result_gp.fun < best_score:
    best_method = "Gaussian Process"
    best_result = result_gp

# Method 2: Gradient Boosted Trees Optimization
print("\n" + "="*60)
print("PHƯƠNG PHÁP 2: GRADIENT BOOSTED TREES OPTIMIZATION")
print("="*60)

results_history = []  # Reset history
result_gbrt = gbrt_minimize(
    func=objective,
    dimensions=dimensions,
    n_calls=50,
    n_initial_points=10,
    random_state=42,
    callback=callback_func
)

print(f"Best GBRT score: {result_gbrt.fun:.4f}")
print("Best GBRT parameters:")
for dim, val in zip(dimensions, result_gbrt.x):
    print(f"  {dim.name}: {val}")
if result_gbrt.fun < best_score:
    best_method = "Gradient Boosted Trees"
    best_result = result_gbrt

# Compare results of methods
print("\n" + "="*60)
print("SO SÁNH KẾT QUẢ")
print("="*60)

methods_comparison = pd.DataFrame({
    'Method': ['Gaussian Process', 'Gradient Boosted Trees'],
    'Best_MAE': [result_gp.fun, result_gbrt.fun],
    'N_calls': [len(result_gp.func_vals), len(result_gbrt.func_vals)]
})

print(methods_comparison.to_string(index=False))


Seed set to 1


PHƯƠNG PHÁP 1: GAUSSIAN PROCESS OPTIMIZATION


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name              | Type                 | Params | Mode 
--------------------------------------------------------------------
0  | loss              | MAE                  | 0      | train
1  | valid_loss        | MAE                  | 0      | train
2  | padder_train      | ConstantPad1d        | 0      | train
3  | scaler            | TemporalNorm         | 0      | train
4  | pdm_blocks        | ModuleList           | 90.6 K | train
5  | preprocess        | SeriesDecomp         | 0      | train
6  | enc_embedding     | DataEmbedding_wo_pos | 2.5 K  | train
7  | normalize_layers  | ModuleList           | 4      | train
8  | predict_layers    | ModuleList           | 2.2 K  | train
9  | projection_layer  | Linear               | 33     | train
10 | out_res_layers    | ModuleList           | 6.6 K  | train
11 | regression_layers | ModuleList           | 2.2 K  | train


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Error in objective function: batch_size should be a positive integer value, but got batch_size=64


Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name              | Type                 | Params | Mode 
--------------------------------------------------------------------
0  | loss              | MAE                  | 0      | train
1  | valid_loss        | MAE                  | 0      | train
2  | padder_train      | ConstantPad1d        | 0      | train
3  | scaler            | TemporalNorm         | 0      | train
4  | pdm_blocks        | ModuleList           | 128 K  | train
5  | preprocess        | SeriesDecomp         | 0      | train
6  | enc_embedding     | DataEmbedding_wo_pos | 2.5 K  | train
7  | normalize_layers  | ModuleList           | 4      | train
8  | predict_layers    | ModuleList           | 2.0 K  | train
9  | projection_layer  | Linear               | 33     | train
10 | out_res_layers    | ModuleList           | 5.2 K  | train
11 | regression_layers | ModuleList           | 2

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Error in objective function: batch_size should be a positive integer value, but got batch_size=64
Best GBRT score: 22.1045
Best GBRT parameters:
  input_size: 46
  max_steps: 288
  learning_rate: 0.0002324672848950434
  batch_size: 32
  e_layers: 3
  d_ff: 256
  dropout: 0.1542703315240835

SO SÁNH KẾT QUẢ
                Method  Best_MAE  N_calls
      Gaussian Process  21.54721       50
Gradient Boosted Trees  22.10450       50


In [23]:
# Compare results of methods
print("\n" + "="*60)
print("SO SÁNH KẾT QUẢ")
print("="*60)

methods_comparison = pd.DataFrame({
    'Method': ['Gaussian Process', 'Gradient Boosted Trees'],
    'Best_MAE': [result_gp.fun, result_gbrt.fun],
    'N_calls': [len(result_gp.func_vals), len(result_gbrt.func_vals)]
})

print(methods_comparison.to_string(index=False))


SO SÁNH KẾT QUẢ
                Method  Best_MAE  N_calls
      Gaussian Process  21.54721       50
Gradient Boosted Trees  22.10450       50


In [24]:

# Train final model with best parameters
print("\n" + "="*60)
print("HUẤN LUYỆN MÔ HÌNH CUỐI CÙNG")
print("="*60)

final_params = dict(zip([dim.name for dim in dimensions], best_result.x))
final_model = TimeMixer(
    h=horizon,
    input_size=final_params['input_size'],
    n_series=1,
    scaler_type='standard',
    max_steps=final_params['max_steps'],
    early_stop_patience_steps=5,
    val_check_steps=5,
    learning_rate=final_params['learning_rate'],
    loss=MAE(),
    valid_loss=MAE(),
    batch_size=final_params['batch_size'],
    e_layers=final_params['e_layers'],
    d_ff=final_params['d_ff'],
    dropout=final_params['dropout']
)

fcst_final = NeuralForecast(models=[final_model], freq='B')
forecasts_final = fcst_final.cross_validation(
    df=df1, 
    val_size=val_len, 
    n_windows=None, 
    test_size=test_len - ((test_len - horizon) % horizon), 
    step_size=horizon
)

# Calculate final metrics
Y_true_final = forecasts_final['y'].values
Y_hat_final = forecasts_final['TimeMixer'].values

MAE_final = mae(Y_true_final, Y_hat_final)
MAPE_final = mape(Y_true_final, Y_hat_final) * 100
RMSE_final = np.sqrt(mse(Y_true_final, Y_hat_final))

# Final results
df_final_results = pd.DataFrame({
    'Model': [f"TimeMixer_{best_method}"],
    'Horizon': [str(horizon)],
    'Data': ["VNINDEX"],
    'MAE': [MAE_final],
    'RMSE': [RMSE_final],
    'MAPE': [MAPE_final]
})

print("\nKẾT QUẢ CUỐI CÙNG:")
print(df_final_results.to_string(index=False))

# Store results
results_df = pd.DataFrame({
    'GP_MAE': result_gp.func_vals,
    'GBRT_MAE': result_gbrt.func_vals
})

# Summary
print("\n" + "="*60)
print("TÓM TẮT")
print("="*60)
print(f"Phương pháp tốt nhất: {best_method}")
print(f"MAE tốt nhất: {best_result.fun:.4f}")
print(f"Số lần gọi hàm: {len(best_result.func_vals)}")
print("\nTham số tối ưu:")
for dim, val in zip(dimensions, best_result.x):
    print(f"  {dim.name}: {val}")

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



HUẤN LUYỆN MÔ HÌNH CUỐI CÙNG



   | Name              | Type                 | Params | Mode 
--------------------------------------------------------------------
0  | loss              | MAE                  | 0      | train
1  | valid_loss        | MAE                  | 0      | train
2  | padder_train      | ConstantPad1d        | 0      | train
3  | scaler            | TemporalNorm         | 0      | train
4  | pdm_blocks        | ModuleList           | 90.6 K | train
5  | preprocess        | SeriesDecomp         | 0      | train
6  | enc_embedding     | DataEmbedding_wo_pos | 2.5 K  | train
7  | normalize_layers  | ModuleList           | 4      | train
8  | predict_layers    | ModuleList           | 2.2 K  | train
9  | projection_layer  | Linear               | 33     | train
10 | out_res_layers    | ModuleList           | 6.6 K  | train
11 | regression_layers | ModuleList           | 2.2 K  | train
--------------------------------------------------------------------
101 K     Trainable params
2.4 K     Non-t

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]


KẾT QUẢ CUỐI CÙNG:
         Model Horizon    Data      MAE      RMSE     MAPE
TimeMixer_None      20 VNINDEX 21.54721 27.890251 1.715207

TÓM TẮT
Phương pháp tốt nhất: None
MAE tốt nhất: 21.5472
Số lần gọi hàm: 1

Tham số tối ưu:
  input_size: 72
  max_steps: 255
  learning_rate: 0.0006021310185147606
  batch_size: 64
  e_layers: 2
  d_ff: 256
  dropout: 0.13777466758976017


# TSMixer

## Bayes

In [11]:
### Bayes
import pandas as pd
import numpy as np
from skopt import gp_minimize, gbrt_minimize
from skopt.space import Integer, Real,Categorical
from skopt.utils import use_named_args
from neuralforecast import NeuralForecast
from neuralforecast.models import TSMixer
from neuralforecast.losses.pytorch import MAE, MSE
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
import warnings
warnings.filterwarnings('ignore')

In [12]:

horizon = 20


# Định nghĩa không gian tìm kiếm cho các siêu tham số TSMixer
dimensions = [
    Integer(low=40, high=80, name='input_size'),
    Integer(low=200, high=500, name='max_steps'),
    Categorical(categories=[32, 64], name='ff_dim'),
    Real(low=1e-4, high=1e-3, prior='log-uniform', name='learning_rate'),  
    Categorical(categories=[32, 64], name='batch_size'),  
    Real(low=0.0, high=0.3, name='dropout'),
]

# Biến toàn cục để lưu kết quả
results_history = []
best_score = float('inf')
best_params = None
best_method = None
best_result = None

# Hàm callback để lưu kết quả tối ưu hóa
def callback_func(result):
    global results_history, best_score, best_params, best_method, best_result
    
    current_score = result.func_vals[-1]
    current_params = dict(zip([dim.name for dim in dimensions], result.x_iters[-1]))
    
    results_history.append({
        'iteration': len(result.func_vals),
        'score': current_score,
        **current_params
    })
    
    if current_score < best_score:
        best_score = current_score
        best_params = current_params.copy()
        best_result = result
        print(f"Điểm tốt nhất mới: {current_score:.4f} tại vòng lặp {len(result.func_vals)}")

@use_named_args(dimensions)
def objective(**params):
    """
    Hàm mục tiêu cho Tối ưu hóa Bayes
    """
    try:
        # Tạo mô hình TSMixer với các siêu tham số
        model = TSMixer(
            h=horizon,
            input_size=params['input_size'],
            max_steps=params['max_steps'],
            n_block=2,
            ff_dim=params['ff_dim'],
            dropout=params['dropout'],
            n_series=1,
            valid_loss=MAE(),
            batch_size=params['batch_size'],           
            learning_rate=params['learning_rate'],
            loss=MAE(),  # Sử dụng MAE làm hàm mất mát
            # Thêm futr_exog_list=['usd_vnd', 'DJI_Close'] nếu được hỗ trợ
        )
        
        # Khởi tạo NeuralForecast
        fcst = NeuralForecast(models=[model], freq='B')
        
        # Thực hiện cross-validation
        print("Hình dạng dữ liệu đầu vào:", df1.shape)
        forecasts = fcst.cross_validation(
            df=df1, 
            val_size=val_len, 
            n_windows=None, 
            test_size=test_len - ((test_len - horizon) % horizon), 
            step_size=horizon
        )
        
        # Tính toán MAE
        Y_true = forecasts['y'].values
        Y_hat = forecasts['TSMixer'].values  # Cập nhật thành 'TSMixer'
        print("Hình dạng dự báo:", forecasts.shape)
        MAE_score = mae(Y_true, Y_hat)
        
        print(f"MAE: {MAE_score:.4f} | Tham số: {params}")
        return MAE_score
        
    except Exception as e:
        print(f"Lỗi trong hàm mục tiêu: {e}")
        return 1000.0  # Phạt cho các lần chạy thất bại


In [13]:

# Phương pháp 1: Tối ưu hóa Gaussian Process
print("="*60)
print("PHƯƠNG PHÁP 1: TỐI ƯU HÓA GAUSSIAN PROCESS")
print("="*60)

results_history = []  # Đặt lại lịch sử
result_gp = gp_minimize(
    func=objective,
    dimensions=dimensions,
    n_calls=50,
    n_initial_points=10,
    acq_func='EI',
    random_state=42,
    callback=callback_func
)

print(f"Điểm GP tốt nhất: {result_gp.fun:.4f}")
print("Tham số GP tốt nhất:")
for dim, val in zip(dimensions, result_gp.x):
    print(f"  {dim.name}: {val}")
if result_gp.fun < best_score:
    best_method = "Gaussian Process"
    best_result = result_gp

# Phương pháp 2: Tối ưu hóa Gradient Boosted Trees
print("\n" + "="*60)
print("PHƯƠNG PHÁP 2: TỐI ƯU HÓA GRADIENT BOOSTED TREES")
print("="*60)

results_history = []  # Đặt lại lịch sử
result_gbrt = gbrt_minimize(
    func=objective,
    dimensions=dimensions,
    n_calls=50,
    n_initial_points=10,
    random_state=42,
    callback=callback_func
)

print(f"Điểm GBRT tốt nhất: {result_gbrt.fun:.4f}")
print("Tham số GBRT tốt nhất:")
for dim, val in zip(dimensions, result_gbrt.x):
    print(f"  {dim.name}: {val}")
if result_gbrt.fun < best_score:
    best_method = "Gradient Boosted Trees"
    best_result = result_gbrt

# So sánh kết quả các phương pháp
print("\n" + "="*60)
print("SO SÁNH KẾT QUẢ")
print("="*60)

methods_comparison = pd.DataFrame({
    'Phương pháp': ['Gaussian Process', 'Gradient Boosted Trees'],
    'MAE tốt nhất': [result_gp.fun, result_gbrt.fun],
    'Số lần gọi': [len(result_gp.func_vals), len(result_gbrt.func_vals)]
})

print(methods_comparison.to_string(index=False))


Seed set to 1


PHƯƠNG PHÁP 1: TỐI ƯU HÓA GAUSSIAN PROCESS
Hình dạng dữ liệu đầu vào: (2556, 3)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type              | Params | Mode 
------------------------------------------------------------
0 | loss          | MAE               | 0      | train
1 | valid_loss    | MAE               | 0      | train
2 | padder_train  | ConstantPad1d     | 0      | train
3 | scaler        | TemporalNorm      | 0      | train
4 | norm          | RevINMultivariate | 2      | train
5 | mixing_layers | Sequential        | 11.5 K | train
6 | out           | Linear            | 1.5 K  | train
------------------------------------------------------------
12.9 K    Trainable params
0         Non-trainable params
12.9 K    Total params
0.052     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Seed set to 1
GPU available: False, used: False


Lỗi trong hàm mục tiêu: batch_size should be a positive integer value, but got batch_size=64
Hình dạng dữ liệu đầu vào: (2556, 3)


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type              | Params | Mode 
------------------------------------------------------------
0 | loss          | MAE               | 0      | train
1 | valid_loss    | MAE               | 0      | train
2 | padder_train  | ConstantPad1d     | 0      | train
3 | scaler        | TemporalNorm      | 0      | train
4 | norm          | RevINMultivariate | 2      | train
5 | mixing_layers | Sequential        | 7.7 K  | train
6 | out           | Linear            | 1.2 K  | train
------------------------------------------------------------
8.9 K     Trainable params
0         Non-trainable params
8.9 K     Total params
0.036     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Lỗi trong hàm mục tiêu: batch_size should be a positive integer value, but got batch_size=32
Điểm GBRT tốt nhất: 23.3953
Tham số GBRT tốt nhất:
  input_size: 41
  max_steps: 473
  ff_dim: 32
  learning_rate: 0.00045975057847321674
  batch_size: 32
  dropout: 0.15602040635334327

SO SÁNH KẾT QUẢ
           Phương pháp  MAE tốt nhất  Số lần gọi
      Gaussian Process     24.793581          50
Gradient Boosted Trees     23.395328          50


In [14]:
# So sánh kết quả các phương pháp
print("\n" + "="*60)
print("SO SÁNH KẾT QUẢ")
print("="*60)

methods_comparison = pd.DataFrame({
    'Phương pháp': ['Gaussian Process', 'Gradient Boosted Trees'],
    'MAE tốt nhất': [result_gp.fun, result_gbrt.fun],
    'Số lần gọi': [len(result_gp.func_vals), len(result_gbrt.func_vals)]
})

print(methods_comparison.to_string(index=False))



SO SÁNH KẾT QUẢ
           Phương pháp  MAE tốt nhất  Số lần gọi
      Gaussian Process     24.793581          50
Gradient Boosted Trees     23.395328          50


In [18]:

# Huấn luyện mô hình cuối cùng với tham số tốt nhất
print("\n" + "="*60)
print("HUẤN LUYỆN MÔ HÌNH CUỐI CÙNG")
print("="*60)

final_params = dict(zip([dim.name for dim in dimensions], best_result.x))

final_model = TSMixer(
            h=horizon,
            input_size=final_params['input_size'],
            max_steps=final_params['max_steps'],
            n_block=2,
            ff_dim=final_params['ff_dim'],
            dropout=final_params['dropout'],
            n_series=1,
            valid_loss=MAE(),
            batch_size=final_params['batch_size'],           
            learning_rate=final_params['learning_rate'],
            loss=MAE(),  # Sử dụng MAE làm hàm mất mát
            # Thêm futr_exog_list=['usd_vnd', 'DJI_Close'] nếu được hỗ trợ
        )

fcst_final = NeuralForecast(models=[final_model], freq='B')
forecasts_final = fcst_final.cross_validation(
    df=df1, 
    val_size=val_len, 
    n_windows=None, 
    test_size=test_len - ((test_len - horizon) % horizon), 
    step_size=horizon
)

# Tính toán các chỉ số cuối cùng
Y_true_final = forecasts_final['y'].values
Y_hat_final = forecasts_final['TSMixer'].values  # Cập nhật thành 'TSMixer'

MAE_final = mae(Y_true_final, Y_hat_final)
MAPE_final = mape(Y_true_final, Y_hat_final) * 100
RMSE_final = np.sqrt(mse(Y_true_final, Y_hat_final))

# Kết quả cuối cùng
df_final_results = pd.DataFrame({
    'Mô hình': [f"TSMixer_{best_method}"],
    'Chân trời': [str(horizon)],
    'Dữ liệu': ["VNINDEX"],
    'MAE': [MAE_final],
    'RMSE': [RMSE_final],
    'MAPE': [MAPE_final]
})

print("\nKẾT QUẢ CUỐI CÙNG:")
print(df_final_results.to_string(index=False))

# Lưu kết quả
results_df = pd.DataFrame({
    'GP_MAE': result_gp.func_vals,
    'GBRT_MAE': result_gbrt.func_vals
})

# Tóm tắt
print("\n" + "="*60)
print("TÓM TẮT")
print("="*60)
print(f"Phương pháp tốt nhất: {best_method}")
print(f"MAE tốt nhất: {best_result.fun:.4f}")
print(f"Số lần gọi hàm: {len(best_result.func_vals)}")
print("\nTham số tối ưu:")
for dim, val in zip(dimensions, best_result.x):
    print(f"  {dim.name}: {val}")

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



HUẤN LUYỆN MÔ HÌNH CUỐI CÙNG



  | Name          | Type              | Params | Mode 
------------------------------------------------------------
0 | loss          | MAE               | 0      | train
1 | valid_loss    | MAE               | 0      | train
2 | padder_train  | ConstantPad1d     | 0      | train
3 | scaler        | TemporalNorm      | 0      | train
4 | norm          | RevINMultivariate | 2      | train
5 | mixing_layers | Sequential        | 4.0 K  | train
6 | out           | Linear            | 840    | train
------------------------------------------------------------
4.8 K     Trainable params
0         Non-trainable params
4.8 K     Total params
0.019     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=473` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]


KẾT QUẢ CUỐI CÙNG:
     Mô hình Chân trời Dữ liệu       MAE      RMSE     MAPE
TSMixer_None        20 VNINDEX 23.395328 30.067503 1.864225

TÓM TẮT
Phương pháp tốt nhất: None
MAE tốt nhất: 23.3953
Số lần gọi hàm: 8

Tham số tối ưu:
  input_size: 41
  max_steps: 473
  ff_dim: 32
  learning_rate: 0.00045975057847321674
  batch_size: 32
  dropout: 0.15602040635334327
